<a href="https://colab.research.google.com/github/DarioMameli/DL_Assignments/blob/main/DL_Assignment3_Clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL Assignment 3
## Group 4
**AUTHORS**:
*   Dario Mameli [dario.mameli@ugent.be]
*   Àngel Masip LLopis [angel.masipllopis@ugent.be]
*   Michele Russo [michele.russo@ugent.be]



This notebook is to be intended as both report and code.

Running on google colab is suggested to make sure all required libraries and packages are present.

#GitHub

Let's clone the repo with the dataset

In [1]:
!git clone https://github.com/dkdk22/DL_Ass3
import sys
# Add the repository to the path
sys.path.insert(1, '/content/DL_Ass3/')

Cloning into 'DL_Ass3'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 13 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (13/13), 30.71 MiB | 15.82 MiB/s, done.
Resolving deltas: 100% (2/2), done.


# Setup

Firstly let's setup the workspace

## Pip Installs

In [2]:
# Install the latest version of TensorFlow, which includes Keras (tf.keras)
#!pip install tensorflow --upgrade
!pip install keras --upgrade --quiet
!pip install keras-tuner --upgrade
!pip install PrettyTable

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.4 MB/s eta 0:00:00


## Packages

Let's import all the necessary functions and packages

In [3]:
from dl_utils import plot_history
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras import regularizers
import matplotlib.gridspec as gridspec
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import nltk
from nltk.corpus import stopwords
import keras_tuner as kt
from keras import backend as backend
from keras.layers import (
    Input,
    Activation,
    BatchNormalization,
    Conv2D,
    Conv2DTranspose,
    Dense,
    Dropout,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
    UpSampling2D
)
from keras.optimizers import SGD, Adam

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import json
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
import gdown
import zipfile
import os
from keras import Model, ops
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
from sklearn.ensemble import GradientBoostingClassifier
from nltk.stem import PorterStemmer
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras.applications as app
from keras.layers import LSTM, Dense, Embedding, Input, GRU, Bidirectional
from sklearn.preprocessing import StandardScaler
import gc

from prettytable import PrettyTable

import zipfile
from skimage.segmentation import mark_boundaries
from sklearn.preprocessing import minmax_scale

import warnings
warnings.filterwarnings('ignore')

#Global variables

To accommodate computational constraints, we have opted to train the model and host the resulting datasets on GitHub. These global variables serve the purpose of indicating whether training or dataset creation is necessary, or if the pre-existing dataset from GitHub should be utilized.

In [4]:
SEED =42
TRAIN_BASELINE = False
TRAIN_MODEL = False
EMBEDDINGS_GENERAL=False

# Load data

Load the dataset

In [5]:
zip_file_path = 'DL_Ass3/review_553850.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref: #Extract the file
    zip_ref.extractall()

with open('review_553850.json', 'r') as json_file: # Open the json file
    data = json.load(json_file)

X = []
y = []

for id, review in data["reviews"].items():
    review_text = review["review"]
    funny_votes = review["votes_funny"]
    X.append(review_text)
    y.append(funny_votes)

##Data preprocessing

We are going to do the same preprocessing as in the other notebook. We are going to eliminate stopwords and also we are going to use steeming, tokenize and pad the sentences.


In [6]:
if EMBEDDINGS_GENERAL:
  nltk.download('stopwords')
  stop_words = set(stopwords.words('english'))
  stemmer = PorterStemmer()
  # Remove stopwords, convert to lowercase and stemming
  X_prepro = []
  for text in X:
      words = text.split()

      filtered_words = [word.lower() for word in words if word.lower() not in stop_words]
      stemmed_tokens = [stemmer.stem(word) for word in filtered_words]
      preprocessed_text = ' '.join(stemmed_tokens)
      X_prepro.append(preprocessed_text)
  # Remove the reviews with 0 words
  X_filtered = []
  y_filtered = []
  for review, label in zip(X_prepro, y):
      if len(review.split()) > 0:
          X_filtered.append(review)
          y_filtered.append(label)
  print("Length of the dataset: "+str(len(X_filtered)))
  # Tokenize
  num_words=10000
  tokenizer = Tokenizer(num_words=num_words)
  tokenizer.fit_on_texts(X_filtered)
  X_sequences = tokenizer.texts_to_sequences(X_filtered)
  # Initialize variables to store information about the longest sequence
  max_length = 0
  max_index = -1

  # Iterate through the list of sequences to have the longes sequence
  for i, sequence in enumerate(X_sequences):
      length = len(sequence)
      # Update max_length and max_index if the current sequence is longer
      if length > max_length:
          max_length = length
          max_index = i

  # Retrieve the longest sequence
  longest_sequence = X_sequences[max_index]

  # Print the longest sequence
  print("The longest sequence:")
  print(longest_sequence)
  print(len(longest_sequence))
  # Padding
  max_sequence_length = len(longest_sequence)
  padded_sequences = pad_sequences(X_sequences, maxlen=max_sequence_length)
  # Convert to np arrays
  scaler = StandardScaler()
  normalized_data = scaler.fit_transform(padded_sequences)
  X = np.array(normalized_data)
  print(X.shape)



In [8]:
if EMBEDDINGS_GENERAL:
  del stop_words
  del stemmer
  del X_prepro
  del text
  del words
  del filtered_words
  del stemmed_tokens
  del preprocessed_text

  del review
  del num_words
  del tokenizer
  del X_sequences
  del max_length
  del max_index
  del sequence
  del longest_sequence
  del padded_sequences
  del scaler
  del normalized_data
  gc.collect()

#Binary classification problem

We can see that this is a really difficoult task to learn because most of the reviews have 0 funny_votes and some of them have a really high value of funny_votes. That's why we are going to change the approach and convert the problem to a binary classification problem, that should be easier to learn to our model.

We are going to convert it by fixing an number of funny_votes N, treating a review as funny or not funny depending on whether it has more or fewer votes than N. In our case N is going to be 1 because of the unbalancement of the dataset.

In [9]:
y = [0 if num == 0 else 1 for num in y_filtered]
y = np.array(y)
counts = [0,0]
counts[0] = np.sum(data == 0)
counts[1] = np.sum(data == 1)
del y_filtered
gc.collect()

NameError: name 'y_filtered' is not defined

##Creating an embedd

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')

##Baseline Model

In [ ]:
baseline_model = GradientBoostingClassifier()

##Best model for classification problem

In [ ]:
class HyperModelBuilderBinary(kt.HyperModel):
    def __init__(self, max_sequence_length, loss_function, max_features):
        self.max_sequence_length = max_sequence_length
        self.loss_function = loss_function
        self.max_features = max_features

    def build_lstm_model(self, hp):
        model = Sequential()
        model.add(Input(shape=(self.max_sequence_length,)))
        model.add(Embedding(input_dim=self.max_features, output_dim=hp.Int('embedding_dim', min_value=128, max_value=128, step=32)))  #fix embedding size?
        model.add(LSTM(units=hp.Int('units', min_value=64, max_value=256, step=32), #https://keras.io/api/layers/recurrent_layers/lstm/
                       dropout=hp.Float('dropout', min_value=0.2, max_value=0.4, step=0.2),
                       recurrent_dropout=hp.Float('dropout', min_value=0.2, max_value=0.4, step=0.2), #reduce two dropouts
                       input_shape=(self.max_sequence_length, 1)))
        model.add(Dense(1, activation="sigmoid"))
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
                      loss=self.loss_function,
                      metrics=["accuracy"])
        return model


    def build_gru_model(self, hp):
        model = Sequential()
        model.add(Input(shape=(self.max_sequence_length,)))
        model.add(Embedding(input_dim=self.max_features, output_dim=hp.Int('embedding_dim', min_value=128, max_value=128, step=32)))
        model.add(GRU(units=hp.Int('units', min_value=64, max_value=256, step=32),   #https://keras.io/api/layers/recurrent_layers/gru/
                      dropout=hp.Float('dropout', min_value=0.1, max_value=0.4, step=0.1),
                      recurrent_dropout=hp.Float('dropout', min_value=0.1, max_value=0.4, step=0.1),
                      input_shape=(self.max_sequence_length, 1)))
        model.add(Dense(1, activation="sigmoid"))

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
                      loss=self.loss_function,
                      metrics=["accuracy"])
        return model

    def build_model(self,hp):
        model_name = hp.Choice('model_name', values=['GRU', 'LSTM'])
        if model_name == 'LSTM':
            return self.build_lstm_model(hp)
        elif model_name == 'GRU':
            return self.build_gru_model(hp)

    def fit(self, hp, model, *args, **kwargs):
        weight_for_0 = 1. / counts[0]
        weight_for_1 = 1. / counts[1]
        class_weight = {0: weight_for_0, 1: weight_for_1}
        return model.fit(X_train, y_train, validation_data=(X_val, y_val),class_weight=class_weight)



In [ ]:
#Assuming y is a numpy array containing labels
#and X is a numpy array containing features
#Indices where y equals 0
indices_y0 = np.where(y == 0)[0]
size_X_y0 = len(indices_y0)

#Indices where y equals 1
indices_y1 = np.where(y == 1)[0]
size_X_y1 = len(indices_y1)

print("Size of X when y = 0:", size_X_y0)
print("Size of X when y = 1:", size_X_y1)

In [ ]:

loss = keras.losses.BinaryCrossentropy(

    name='binary_crossentropy'
)
# Define hyperparameters for the tuner
hypermodel_builder = HyperModelBuilderBinary(max_sequence_length=max_sequence_length,
                                       loss_function=loss, max_features=10000)

# Initialize tuner
tuner = kt.BayesianOptimization(hypermodel_builder.build_model,
                        objective='val_loss',
                        max_trials=1,
                        executions_per_trial=1, # The documentation says that makes things faster
                        overwrite=True,
                        max_consecutive_failed_trials=5,
                        directory='tuner_results',
                        project_name='regression_tuning')

# Summarize the search space
tuner.search_space_summary()

##Training & Evaluating

In [ ]:
def compute_class_accuracies(X_data, y_data, model):
  # The predicted probabilities for each class
  y_pred_probs = model.predict(X_data)

  # The corresponding predicted labels
  y_pred_labels = np.argmax(y_pred_probs, axis=0)

  # The corresponding true labels
  y_true_labels = np.argmax(y_data, axis=0)

  # Compute confusion matrix
  cm = confusion_matrix(y_true_labels, y_pred_labels)

  # Compute accuracies and weights for each class
  class_accuracies = []
  for class_label in range(len(cm)):
      class_accuracy = cm[class_label, class_label] / np.sum(cm[class_label, :])
      class_accuracies.append(class_accuracy)

  # Print the classification report
  print(classification_report(y_true_labels, y_pred_labels))

  return class_accuracies

In [ ]:
import numpy as np

def balance_classes(X_train, y_train):
    # Find indices of examples for each class
    indices_class_0 = np.where(y_train == 0)[0]
    indices_class_1 = np.where(y_train == 1)[0]

    # Calculate the number of examples in each class
    num_class_0 = len(indices_class_0)
    num_class_1 = len(indices_class_1)

    # Randomly select a subset of examples from class 0 to match the number of examples in class 1
    if num_class_0 > num_class_1:
        # Randomly shuffle indices of class 0 examples
        np.random.shuffle(indices_class_0)
        # Keep only enough examples from class 0 to match the number of examples in class 1
        indices_class_0 = indices_class_0[:num_class_1]
    elif num_class_1 > num_class_0:
        # Randomly shuffle indices of class 1 examples
        np.random.shuffle(indices_class_1)
        # Keep only enough examples from class 1 to match the number of examples in class 0
        indices_class_1 = indices_class_1[:num_class_0]

    # Combine the balanced indices
    balanced_indices = np.concatenate([indices_class_0, indices_class_1])

    # Shuffle the combined indices
    np.random.shuffle(balanced_indices)

    # Return the balanced subset of data
    return X_train[balanced_indices], y_train[balanced_indices]


In [ ]:
#balanced_X_train, balanced_y_train = balance_classes(X, y)
X, _, y, _ = train_test_split(X_filtered,y, test_size=0.8, shuffle=True, random_state=SEED, stratify=y)

##Create the embeddings

In [ ]:
import tensorflow as tf
import numpy as np
from sentence_transformers import SentenceTransformer

if EMBEDDINGS_GENERAL:
  # Load the BERT model
  model = SentenceTransformer('bert-base-nli-mean-tokens')

  # Define a function to encode a sentence using the model
  def encode_sentence(sentence):
      return model.encode(sentence)

  # Number of GPUs to use
  num_gpus = len(tf.config.list_physical_devices('GPU'))

  # Check if GPU is available
  if num_gpus == 0:
      print("No GPU available. Using CPU.")
  else:
      print("Number of GPUs available:", num_gpus)

  # Distribute the work across multiple GPUs using MirroredStrategy
  strategy = tf.distribute.MirroredStrategy()


  # Encode sentences using the model in parallel
  encoded_sentences = []
  i=0
  for sentence in X:
      i=i+1
      encoded_sentence = encode_sentence(sentence)
      encoded_sentences.append(encoded_sentence)
      print(i)

  # Convert the list of encoded sentences into a numpy array
  X = np.array(encoded_sentences)

  # Print the shape of the encoded sentences
  print("Shape of encoded sentences:", X.shape)
  # Assuming X_train is your encoded dataset
  np.save('/content/X.npy', X)
else:
  X = np.load('/content/X.npy')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, shuffle=True, random_state=SEED, stratify=y)

In [ ]:
print(X_train.shape)
count_1=0
count_0=0
for sample in y_train:
  if sample==0:
    count_0=1+count_0
  else:
    count_1=1+count_1
print("class 1 ", count_1)
print("class 0", count_0)

In [ ]:
#if TRAIN_BASELINE:
baseline_model.fit(X_train, y_train)
joblib.dump(baseline_model, '/content/DL_Ass3/baseline_model_cla.pkl')
#else:
baseline_model = joblib.load('/content/DL_Ass3/baseline_model_cla.pkl')

In [ ]:
y_pred = baseline_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
  # Print the classification report
print(classification_report(y_test, y_pred))

We can see that the model is only predicting 0

Train or model

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True,  random_state=SEED, stratify=y_train)

In [ ]:
del y
del X
gc.collect()

In [ ]:
import pickle
if TRAIN_MODEL:
  # Early stop to prevent overfitting
  stop_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=4,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,
  )

  # Perform hyperparameter search
  tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=1, batch_size=256, callbacks=[stop_early])
  # Get the best hyperparameters
  best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
  # Define the file name where you want to save the hyperparameters
  file_name = "/content/DL_Ass3/best_hps_cla.json"
  #Save the hp
  with open("/content/DL_Ass3/best_hps_cla.pkl", "wb") as f:
    pickle.dump(best_hps, f)
  # Build the model
  best_model = tuner.hypermodel.build(best_hps)
  print(best_model)
  # Fit the model
  history = best_model.fit(X_train, y_train, epochs=50, batch_size=256, validation_data=(X_val, y_val), callbacks=[stop_early])
  # Save the model
  best_model.save_weights('/content/DL_Ass3/best_model_cla.weights.h5')
  plot_history(history)
else:
  with open("/content/DL_Ass3/best_hps_cla.pkl", "wb") as f:
    best_hps = pickle.load(f)
  best_model = tuner.hypermodel.build(best_hps)
  best_model.load_weights('/content/DL_Ass3/best_model_cla.weights.h5')


In [ ]:
with open("/content/DL_Ass3/best_hps_cla.pkl", "rb") as f:
  best_hps = pickle.load(f)
best_model = tuner.hypermodel.build(best_hps)